In [11]:
run_names = ["a01_20180630-145845", "a01_20180702-163212", "a01_20180702-162702", "a01_20180703-191614"]


In [12]:
from pathlib import Path
from PIL import Image

In [13]:
base_dir = Path(r"E:\Trackmania Data\data")
img_paths = [base_dir / Path(run) / Path("screenshots") for run in run_names]

In [15]:
img_paths

[WindowsPath('E:/Trackmania Data/data/a01_20180630-145845/screenshots'),
 WindowsPath('E:/Trackmania Data/data/a01_20180702-163212/screenshots'),
 WindowsPath('E:/Trackmania Data/data/a01_20180702-162702/screenshots'),
 WindowsPath('E:/Trackmania Data/data/a01_20180703-191614/screenshots')]

In [16]:
for path in img_paths:
    for e in path.iterdir():
        im = Image.open(e).crop(box=(0, 0, 800, 320)).resize((400, 160), Image.LANCZOS)
        im.save(e)